# 14 · Where Does Chaos Reside? — Testing Alternative Operators

**Observational record associated with the book**  
*Discovering Chaos in Prime Numbers — Computational Investigations through the Euler Mirror*  
© Alvaro Costa, 2025

This notebook is part of a canonical sequence of computational records.  
It introduces **no new hypotheses, conjectures, or interpretative models**.

Its sole purpose is to **record** the behaviour of arithmetical structures under an explicit,  
deterministic, and reproducible observation regime.

The complete conceptual interpretation is presented in the book.  
This notebook documents only the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, adapted redistribution, or independent commercial use are not permitted.


---

## 1. The Final Doubt: Is the Operator an Artefact?

Our journey has revealed a clear statistical duality in the “music” of the primes, made visible through the operator

$
M_{ij} \propto \cos\,\bigl(f_i \ln x_j\bigr) + \cos\,\bigl(f_j \ln x_i\bigr),
$

where $ f_i = \Delta_\pi(x_i) $.

However, one final and crucial question remains open:  
is this duality — Poisson at the linear scale and GOE at the logarithmic scale — an **intrinsic property of the arithmetical signal of the primes**, or could it be an accidental artefact of the specific functional choice of the **cosine**?

In other words: what if the “music” we are hearing does not reside in the primes themselves, but rather in the mathematical *instrument* used to probe them?

To address this doubt rigorously, we must change the instrument while keeping everything else fixed.

---

## 2. The Experiment: Replacing the Matrix Kernel

The use of the cosine is not arbitrary. It arises naturally as the real part of a complex phase:

$
\cos(\theta) = \mathrm{Re}\,\left(e^{i\theta}\right).
$

The most direct and conceptually natural generalisation is therefore to abandon the real projection and work instead with the full complex phase.

We thus introduce an alternative operator, based on a **phase kernel**:

$
M'_{ij} = e^{\,i\, f_i \ln x_j}.
$

This new operator preserves all the essential elements of the problem:

* the same arithmetical signal $ \Delta_\pi(x) $;
* the same dependence on the logarithmic scale;
* the same interaction structure between indices.

What changes is solely the functional form of the kernel.

**Operational hypothesis:**
if the Poisson/GOE duality is a structural phenomenon, arising from the interaction between the prime signal and the scale of observation — rather than from the cosine function itself — then the phase operator should exhibit exactly the same statistical behaviour:

* **Poisson** statistics at the linear scale;
* statistics compatible with **GOE** at the logarithmic scale.

---

## 3. The Operator Laboratory

The code cell below implements this robustness test.

A selector has been introduced that allows one to switch between:

* the **Cosine Kernel** (the original operator);
* the **Phase Kernel** (the alternative operator),

while keeping fixed:

* the input signal $ \Delta_\pi(x) $;
* the observation regimes (linear and logarithmic);
* the statistical protocol applied to the spectrum.

Compare the results side by side.
If the Poisson/GOE signature remains unchanged under the replacement of the kernel, the origin of the phenomenon will have been isolated unequivocally:  
it does not reside in the operator, but in the arithmetical structure observed at the correct scale.


In [1]:
# Requirements: pandas, matplotlib, numpy, ipywidgets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Data and Matrix Generation Functions (from the previous chapter) ---
def generate_pi_data(n: int) -> np.ndarray:
    """Generates an array containing all primes up to n using an optimised sieve."""
    if n < 2:
        return np.array([], dtype=np.int64)

    size = (n - 1) // 2
    sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2

    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3
            start = (p * p - 3) // 2
            sieve[start::p] = False

    indices = np.where(sieve)[0]
    odd_primes = 2 * indices + 3

    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))


def get_delta_pi_for_points(x_points, primes):
    """Computes Δπ(x) for an array of x-points using a precomputed list of primes."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2


# --- Matrix Generation Functions ---
def generate_cosine_matrix(fx_values, x_values):
    """Generates the matrix M using the cosine kernel."""
    fx = fx_values.astype(np.float64)
    x = x_values.astype(np.float64)

    x[x <= 0] = 1e-12
    logx = np.log(x)

    C = np.cos(np.outer(fx, logx))
    M = C + C.T

    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev

    return 0.5 * (M + M.T)


def generate_phase_matrix(fx_values, x_values):
    """Generates the matrix M using the complex phase kernel."""
    fx = fx_values.astype(np.float64)
    x = x_values.astype(np.float64)

    x[x <= 0] = 1e-12
    logx = np.log(x)

    # --- Construct the complex phase matrix ---
    U = np.exp(1j * np.outer(fx, logx))
    M = U + U.conj().T

    std_dev = M.std()
    if std_dev > 0:
        M = M - M.mean()
        M = M / std_dev

    return M


def local_normalise_spacings(lam, alpha=0.10, w=21):
    """Applies local unfolding to the level spacings."""
    N = lam.size
    k0, k1 = int(alpha * N), int((1 - alpha) * N)

    lam_bulk = np.sort(lam)[k0:k1]
    s = np.diff(lam_bulk)
    s = s[s > 0]

    if len(s) < w:
        return s / s.mean() if s.mean() > 0 else s

    w = int(w)
    if w % 2 == 0:
        w += 1

    pad = w // 2
    s_padded = np.pad(s, (pad, pad), mode='reflect')
    local_mean = np.convolve(s_padded, np.ones(w) / w, mode='valid')
    local_mean[local_mean == 0] = 1.0

    return s / local_mean


# --- Main Interactive Function ---
def operator_robustness_lab(N=2048, log_X0=8, span=2.4, kernel_type='Cosine'):

    X0 = int(10 ** log_X0)

    # --- Data Preparation ---
    max_x_log = int(np.ceil(X0 * np.exp(span / 2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)

    pi_x_full = generate_pi_data(max_x_needed)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

    # Input signals (identical for both operators)
    fx_linear = get_delta_pi_for_points(np.arange(X0, X0 + N), pi_x_full)
    fx_log = get_delta_pi_for_points(
        np.exp(np.linspace(np.log(X0) - span / 2, np.log(X0) + span / 2, N)),
        pi_x_full
    )

    # --- Kernel Selection ---
    if kernel_type == 'Cosine':
        matrix_generator = generate_cosine_matrix
    elif kernel_type == 'Phase':
        matrix_generator = generate_phase_matrix
    else:
        print("Invalid kernel.")
        return

    # --- Left Panel: Linear Scale ---
    print(f"\n--- Processing Linear Scale with {kernel_type} Kernel ---")
    x_linear = np.arange(X0, X0 + N)
    M_linear = matrix_generator(fx_linear, x_linear)
    lam_linear, _ = np.linalg.eigh(M_linear)
    s_unfolded_linear = local_normalise_spacings(lam_linear)

    # --- Right Panel: Logarithmic Scale ---
    print(f"\n--- Processing Logarithmic Scale with {kernel_type} Kernel ---")
    x_log = np.exp(np.linspace(np.log(X0) - span / 2, np.log(X0) + span / 2, N))
    M_log = matrix_generator(fx_log, x_log)
    lam_log, _ = np.linalg.eigh(M_log)
    s_unfolded_log = local_normalise_spacings(lam_log)

    # --- Plots ---
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    pdf_poisson = np.exp(-s_grid)

    ax = axes[0]
    ax.hist(s_unfolded_linear, bins=75, density=True, alpha=0.75, label='Data (Linear)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('a) Linear Scale', fontsize=14)
    ax.set_xlabel('s (Normalised Spacing)')
    ax.set_ylabel('Density')
    ax.set_xlim(0, 4)
    ax.legend(loc='upper right')

    ax = axes[1]
    ax.hist(s_unfolded_log, bins=75, density=True, alpha=0.75, label='Data (Log)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('b) Logarithmic Scale', fontsize=14)
    ax.set_xlabel('s (Normalised Spacing)')
    ax.set_xlim(0, 4)
    ax.legend(loc='upper right')

    fig.suptitle(f"Operator Robustness Analysis ({kernel_type} Kernel)",
                 fontsize=18, weight='bold')
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# --- Interactive Widget ---
interact(
    operator_robustness_lab,
    N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
    log_X0=widgets.IntSlider(min=5, max=8, step=1, value=8,
                             description='X₀ = 10^', continuous_update=False),
    span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=2.4,
                             description='Span (log):'),
    kernel_type=widgets.ToggleButtons(options=['Cosine', 'Phase'],
                                      description='Kernel:')
);


interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

---

## 4. Analysis of the Result: Robustness of the Internal Structure

The outcome of the test with the **Phase Kernel** is unambiguous: it reproduces exactly the same statistical behaviour observed with the **Cosine Kernel**.

Across both observation scales, the pattern remains stable:

* on the **linear scale**, the spectrum exhibits **Poisson-type** statistics;
* on the **logarithmic scale**, **GOE** statistics emerge robustly.

This result is neither trivial nor automatic. Although the kernel is complex-valued, the spectral analysis remains effectively in the orthogonal regime. It constitutes direct evidence that the phenomenon observed throughout this work **is not an artefact of the specific kernel function chosen**.

A physical analogy is instructive. Fracturing a quartz crystal along different directions does not alter the fundamental geometry of the faces that are revealed. The cleavage planes are not created by the act of breaking the crystal — they are already inscribed in its internal structure. The experiment merely makes them visible.

The same situation arises here. The **Cosine Kernel** and the **Phase Kernel** represent two distinct mathematical “cleavages” applied to the same underlying system. The fact that both reveal the **same statistical duality** — Poisson on the linear scale and GOE on the logarithmic scale — demonstrates that the origin of the phenomenon does not reside in the specific operator.

The observed statistical duality **does not live in the operator**.

It emerges, robustly and systematically, from the interaction between two fundamental elements:

* the **intrinsic properties of the arithmetic signal associated with the prime numbers**;
* the **geometric scale** at which that signal is observed.

The operator acts merely as a probing instrument. It does not create the statistical structure; it only exposes it.

What this experiment therefore establishes is a structural result:  
the statistical music of the primes — local order and universal chaos — is a deep property of the system, as intrinsic as the hexagonal symmetry of a snowflake.

The instrument may change.  
The structure remains.
